In [ ]:
import eia_model as em
import test as t
import plotly.plotly as py
from plotly.graph_objs import *
from plotly import tools
from datetime import datetime, timedelta

# Create a weather forecast for SeaTac
http://openweathermap.org/

In [ ]:
# Use GetWeatherForecast to pull SeaTac weather forecast from OpenWeatherMap
forecast = em.GetWeatherForecast(t.owm_api, t.city_id, t.units)

In [ ]:
# create interpolated hourly forecast and 3 hour forecast to compare
temps_one_hour = forecast.hourly_temps[:28]
temps_three_hour = forecast.dataframe[:10]

In [ ]:
# plot hour against 3 hour
fig = tools.make_subplots(rows=2)
fig.append_trace(Scatter(
        x=temps_one_hour.index,
        y=temps_one_hour['temp'],
        name='one hour',
        mode = 'lines+markers'), 1, 1
)
fig.append_trace(Scatter(
        x=temps_three_hour.index,
        y=temps_three_hour['temp'],
        name='three hour', mode = 'lines+markers'), 2, 1
)
py.iplot(fig, filename='SeaTac Weather Forecast')

# Pull SeaTac weather station history from NCDC
http://www7.ncdc.noaa.gov/rest/

In [ ]:
# create date index to pull weather history
date2 = temps_one_hour.index.max()- timedelta(days=7)
date1 = date2 - timedelta(hours=28)

In [ ]:
# function to convert datetime to string %Y%m%d%H%M format for NCDC
def convert_date_ncdc(date):
    return datetime.strftime(date, '%Y-%m-%d %H:%M')

In [ ]:
# Use GetWeather to pull hourly data from SeaTac weather station
start, end = convert_date_ncdc(date1), convert_date_ncdc(date2)
history = em.GetWeatherHistory(t.ncdc_api, t.station, t.variable, start, end)

In [ ]:
# plot hour against last week
fig = tools.make_subplots(rows=2)
fig.append_trace(Scatter(
        x=temps_one_hour.index,
        y=temps_one_hour['temp'],
        name='one hour',
        mode = 'lines+markers'), 1, 1
)
fig.append_trace(Scatter(
        x=history.dataframe.index,
        y=history.dataframe['temp'],
        name='last week',
        mode = 'lines+markers'), 2, 1
)
py.iplot(fig, filename='SeaTac Weather Forecast vs Last Week')

# Import Seattle City Light load data
http://www.eia.gov/opendata/date

In [ ]:
# Create function to convert date to EIA date format
def convert_date_eia(date):
    date = local2utc(date)
    return datetime.strftime(date, '%Y%m%dT%HZ')

# Need to define some functions to convert from to and from PCT to GMT
def local2utc(date):
    return date + timedelta(hours=7)

def utc2local(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return datetime.strftime(date - timedelta(hours=7), '%Y-%m-%d %H:%M:%S')

In [ ]:
# Use GetEnergy to pull hourly data from EIA API for Seattle City Light
start, end = convert_date_eia(date1), convert_date_eia(date2)
scl_load = em.GetEnergy(t.eia_api, t.sid, start, end)

In [ ]:
#Plot load data against weather history
trace1 = Scatter(
    x=[utc2local(x) for x in scl_load.dataframe[t.sid].index],
    y=scl_load.dataframe[t.sid]['values'],
    mode = 'lines+markers',
    name='scl_load',
)

trace2 = Scatter(
    x=history.dataframe.index,
    y=history.dataframe['temp'],
    mode = 'lines+markers',
    name = 'temp',
    yaxis='y2'
)

data = [trace1, trace2]

layout = Layout(
    title='SCL Load vs SeaTac Temperature',
    yaxis=dict(
        title='Load MWhs',
        titlefont=dict(
            color='rgb(0,102,204)'
        ),
        tickfont=dict(
            color='rgb(0,102,204)'
        )
    ),
    yaxis2=dict(
        title='Temp F',
        titlefont=dict(
            color='rgb(255,140,0'
        ),
        tickfont=dict(
            color='rgb(255,140,0'
        ),
        overlaying='y',
        side='right'
    )
)
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='multiple-axes-double')